In [ ]:
%%capture --no-display
%pip install requests
%pip install python-dotenv

UsageError: Cell magic `%%pip` not found (But line magic `%pip` exists, did you mean that instead?).


In [2]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def read_input_url(url):
    headers = {
        'Cookie': f'session={os.getenv("SESSION_COOKIE")}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.text

In [3]:
"""Day 1 input"""
day_1_input_url = "https://adventofcode.com/2024/day/1/input"
content = read_input_url(day_1_input_url)
 
left = []
right = []
for line in content.splitlines():
    left_num, right_num = line.split("   ")
    left.append(int(left_num))
    right.append(int(right_num))

In [4]:
"""Solution to Day 1 Puzzle 1"""
def solution_1_1(left, right):
    similarity_score = 0
    for i in range(len(left)):
        similarity_score += abs(left[i] - right[i])
    return similarity_score

solution_1_1(left, right)

29037157

In [5]:
"""Solution to Day 1 Puzzle 2"""
def solution_1_2(left, right):
    similarity_score = 0
    for i in range(len(left)):
        left_num = left[i]
        occurences = right.count(left_num)
        similarity_score += left_num * occurences if occurences > 0 else 0
    return similarity_score

solution_1_2(left, right)

19678534

In [6]:
"""Day 2 input"""
day_2_input_url = "https://adventofcode.com/2024/day/2/input"
content = read_input_url(day_2_input_url)

# A report is a list of levels seperated by new line
# A level is a number, seperasted by space

list_of_reports = []
for line in content.splitlines():
    report = line.split(" ")
    report = [int(level) for level in report]
    list_of_reports.append(report)

In [15]:
"""Soltion to Day 2 Puzzle 1"""

def is_ascending(report):
    return all(report[i] <= report[i+1] for i in range(len(report)-1))
    
def is_descending(report):
    return all(report[i] >= report[i+1] for i in range(len(report)-1))
    
def diff_within(report, min, max):
    for i in range(len(report)-1):
        diff = abs(report[i] - report[i+1])
        if diff < min or diff > max:
            return False
            
    return True

def is_safe(report):
    if not (is_ascending(report) or is_descending(report)):
        return False
    return diff_within(report, 1, 3)

count_safe_reports = 0
for report in list_of_reports:
    if is_safe(report):
        count_safe_reports += 1

count_safe_reports

359

In [19]:
"""Soltion to Day 2 Puzzle 2"""

def is_safe_with_tolerance_of_one_level_removed(report):
    for i in range(len(report)):
        new_report = report[:i] + report[i+1:]
        if is_safe(new_report):
            return True
    return False

count_safe_reports_with_tolerance_of_one_removed = 0
for report in list_of_reports:
    if is_safe_with_tolerance_of_one_level_removed(report):
        count_safe_reports_with_tolerance_of_one_removed += 1

count_safe_reports_with_tolerance_of_one_removed

418